In [10]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import openai
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
llm= OpenAI(temperature = 0.2)


# get sota from one review article,this will serve as first world model

In [113]:
llm('Split this sentence into coherent chunks:The main chronic autoimmune neuropathies include chronic inflammatory demyelinating polyneuropathy (CIDP), multifocal motor neuropathy (MMN), and anti‐myelin‐associated glycoprotein (MAG) demyelinating neuropathy. On the basis of randomized controlled studies, corticosteroids, intravenous immunoglobulin (IVIg), and plasmapheresis provide short‐term benefits in CIDP. MMN responds only to IVIg. Because in MMN and CIDP, IVIg infusions are required every 3–6 weeks to sustain benefits or long‐term remissions, there is a need for “IVIg‐sparing” agents. In CIDP, immunosuppressive drugs, such as azathioprine, cyclosporine, methotrexate, mycophenolate, and cyclophosphamide, are used, but controlled trials have not shown that they are effective. Controlled trials have also not shown benefit to any agents in anti‐MAG neuropathy. However, clinicians use many immunosuppressive drugs in both settings, but all have potentially serious side effects and are only effective in some patients. Thus, there is a need for new therapies in the inflammatory and paraproteinemic neuropathies. New agents targeting T cells, B cells, and transmigration and transduction molecules are discussed as potential treatment options for new trials. The need for biomarkers that predict therapeutic responses or identify patients with active disease is emphasized, and the search for better scoring tools that capture meaningful changes after response to therapies is highlighted.')


'\n\n1. The main chronic autoimmune neuropathies include chronic inflammatory demyelinating polyneuropathy (CIDP), multifocal motor neuropathy (MMN), and anti‐myelin‐associated glycoprotein (MAG) demyelinating neuropathy. \n2. On the basis of randomized controlled studies, corticosteroids, intravenous immunoglobulin (IVIg), and plasmapheresis provide short‐term benefits in CIDP. \n3. MMN responds only to IVIg. \n4. Because in MMN and CIDP, IVIg infusions are required every 3–6 weeks to sustain benefits or long‐term remissions, there is a need for “IVIg‐sparing” agents. \n5. In CIDP, immunosuppressive drugs, such as azathioprine, cyclosporine, methotrexate, mycophenolate, and cyclophosphamide, are used, but controlled trials have not shown that they are effective. \n6. Controlled trials have also not shown benefit to any agents in anti‐MAG neuropathy. \n7. However, clinicians use many immunosuppressive'

In [ ]:
sota_template = """
CIDP: 
Symptoms:
Treatments:

"""

## put sota as context for prompt - try summarizing review article first; get llm to chunk it and write chunks to vector db


## find paragraphs with biological info

In [ ]:
# prompt definition
para = "We found novel previously uncharacterized genes of relevance to CIDP or VAS pathogenesis. Of particular interest in CIDP were tachykinin precursor 1, which may be involved in pain mediation, stearoyl-co-enzyme A(CoA) desaturase, which may be a marker for remyelination, HLA-DQB1, CD69, an early T-cell activation gene, MSR1, a macrophage scavenger receptor, and PDZ and LIM domain 5 (PDLIM5), a factor regulating nuclear factor(NF)-kappa B activity. Allograft inflammatory factor-1 (AIF-1), a modulator of immune response was upregulated both in CIDP and VAS."


In [ ]:
extract_template = """
Given the following paragraph: {paragraph}, extract the relevant {entities}
"""

## extract genes, proteins, molecules from paragraph

In [ ]:
prompt_template = PromptTemplate(
    input_variables = ["paragraph", "entities"],
    template = extract_template
)

In [ ]:
# define chain
extract_chain = LLMChain(
    llm=llm, prompt = prompt_template, output_key = "list",verbose=False
)

In [ ]:
output = extract_chain({"paragraph":para, "entities":"genes"})
ans = output.get('list')

In [ ]:
# function to only keep entities within parentheses
def keep_parens(sentence):
    new_sentence = ''
    if '(' in sentence:
        loc = sentence.find('(')
        for i in range(loc + 1, len(sentence)):
            if sentence[i] == ')':
                return new_sentence
            else:
                new_sentence += sentence[i]
                continue
        return new_sentence
    else:
        return sentence
keep_parens('PDZ and LIM domain 5')                 

In [14]:
# function to remove entities with parentheses
def remove_paras(para):
        sentence = para
        return_sentence = sentence
        new_sentence = ''
        if '(' in sentence:
            loc = sentence.find('(')
            for i in range(loc, len(sentence)):
                if sentence[i] == ')':
                    new_sentence += sentence[i]
                    break
                else:
                    new_sentence += sentence[i]
                    continue
            return_sentence = sentence.replace(new_sentence, '')
            return return_sentence
        else: return return_sentence


def extract_entities(para,entities):
    llm = OpenAI(temperature=0.0)
    extract_template = """
        Given the following paragraph: {paragraph}, extract the relevant {entities}
        """
    prompt_template = PromptTemplate(
        input_variables=["paragraph", "entities"],
        template=extract_template
    )
    extract_chain = LLMChain(
        llm=llm, prompt=prompt_template, output_key="list", verbose=False
    )

    output = extract_chain(
        {"paragraph": para, "entities": entities})
    return output.get('list')


In [15]:
extract_entities('PDZ and LIM domain 5 (PDLIM5), a factor regulating nuclear factor(NF)-kappa B activity. Allograft inflammatory factor-1 (AIF-1), a modulator of immune response was upregulated both in CIDP and VAS.',entities='genes')

'\nAnswer: PDLIM5, NF-kappa B, AIF-1'

## for each gene found, put in genes list

In [ ]:
#ans = clean_output.get('list')
ans = ans.strip('\n')
ans = ans.split(',')
print(ans)

In [ ]:
# put items in ans into genes list
genes_raw = []
for i in range(len(ans)):
    cleaned = keep_parens(ans[i])
    genes_raw.append(cleaned)
print(genes_raw)

## translate genes -> UMLS
* look up genes from UMLS API, get correct definition
* find 1-hop and 2-hop neighbors from SEMMED

In [ ]:
import requests

### for each gene, look up in UMLS, put definition in genes_cui
"raw": "https://uts-ws.nlm.nih.gov/rest/search/current?apiKey={{api-key}}&string=renal tubular acidosis",


In [ ]:
## for each gene, look up in UMLS, put CUI in genes_cui
genes_cui = []
for i in range(len(genes_raw)):
    url = "https://uts-ws.nlm.nih.gov/rest/search/current?apiKey=4cd35c0d-838b-4a0d-90ca-446f3b29a7dc&string=" + genes_raw[i]
    print(f'fetching {genes_raw[i]}')
    response = requests.get(url)
    data = response.json()
    cui = data.get('result').get('results')[0].get('ui')
    genes_cui.append(cui)
print(f'CUIs: {genes_cui}')


## check how to search medikanren? 
* which format does it take - CUIs? up/downregulate; # hops?
* do CUIs generalize to other databases - e.g. SEMMED? where is CUI useful? -> seems like just UMLS
* is semmed just paper citations?
* are there databases with lots of molecular interactions built in?->Kegg, BioGrid, Reactome, WikiPathways, BioCyc, SignaLink, Signor, BioCarta, Pho
* is the thing you want gene regulation? decrease protein expression?